In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR='D:\Temp\MachineLearning\data\cifar-10-batches-py'

class   CifarData:
    def __init__(self,filelist,shuffle):
        all_data=[]
        all_labels=[]
        #循环读取多个批次的数据集文件
        for filename in filelist:
            data,labels=self.load_data(filename)      
            all_data.append(data)
            all_labels.append(labels)
        # 将所有data以纵向的方式合并到_data
        self._data=np.vstack(all_data)
        # 数据归一化处理
        # self._data=self._data/127.5 - 1
        # 将所有label以横向方式合并到_label
        self._labels=np.hstack(all_labels)
        # 数据中样本个数
        self._example_num=self._data.shape[0]
        print(self._example_num)
        # 定位数据集遍历的位置指针
        self._indicator=0
        # 根据传入的shuffle决定是否对数据进行打乱
        self._shuffle=shuffle
        if self._shuffle:
            self.shuffle_data()
                 
    def load_data(self,filename):
        """读取一个批文件中的数据"""
        with open(filename,'rb') as f:
            data=pickle.load(f,encoding='bytes')
            return data[b'data'],data[b'labels']
        
    def shuffle_data(self):
        # 返回一个打乱的序号列表，例如将0~5 随机打乱为 [5,3,2,4,0,1]
        shuffle_list=np.random.permutation(self._example_num)
        # 根据随机列表重排数据和对应的标签值
        self._data=self._data[shuffle_list]
        self._labels=self._labels[shuffle_list]
        
    def next_batch(self,batch_size ):
        '''返回当前指针self._indicator之后batch_size个数据'''
        end_indicator=self._indicator+batch_size
        # 如果相加结果超过了样本个数
        if end_indicator>self._example_num:
            # 如果可以洗牌，打乱数据并从0开始重新取数据
            if self._shuffle:
                self.shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("没有更多数据了！")    
        # 返回从_indicator到end_indicator的data和labels，并将指针后移到end_indicator
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels

# 拼接训练数据的路径名        
train_file=[os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range (1,6)]
test_file=[os.path.join(CIFAR_DIR,'test_batch')]
# 创建训练数据对象
train_data=CifarData(train_file,True)     

batch_size=20
x=tf.placeholder(tf.float32,[batch_size,3072])
y=tf.placeholder(tf.int64,[batch_size])
train_flag=tf.placeholder(tf.bool,[])
# 将一维向量x转化为具有三通道的多维图片，并交换向量通道
x_img=tf.reshape(x,[-1,3,32,32])
x_img=tf.transpose(x_img,[0,2,3,1])

# 将一批batch_size张图片在第一维上切分为单张图片
img_arr=tf.split(x_img,batch_size,axis=0)
res_arr=[]
# 遍历每个图片对其进行处理
for img in img_arr:
    # 将单张四维的图片[1,32,32,3]处理成三维[32,32,3]
    img=tf.reshape(img,[32,32,3])
    # 对单张图片进行图像增强
    img_flip=tf.image.random_flip_left_right(img)     # 翻转图片
    img_bright=tf.image.random_brightness(img_flip,max_delta=63)    # 随机调整亮度
    img_contrast=tf.image.random_contrast(img_bright,lower=0.2, upper=1.8)  # 调整对比度
    # 将增强后的图片再变回原来的四维格式
    img=tf.reshape(img_contrast,[1,32,32,3])
    # 将每个处理后的图片放在一个数组
    res_arr.append(img)
# 将处理后的单个图片重新拼接在一起    
img_aug=tf.concat(res_arr,axis=0)
# 归一化处理
img_input=img_aug/127.5-1

def convert_wrapper(x_input,name,train_flag,activation=tf.nn.relu,initializer=None):
    conv = tf.layers.conv2d(x_input,   #输入
                       32,      #输出通道数
                       (3,3),   #卷积核大小
                       padding='same',          #填充使图小大小不变
                       activation=None,   #激活函数
                       kernel_initializer=initializer,  # 初始化方法
                       name=name             
                       )
    bn = tf.layers.batch_normalization(conv,training=train_flag)
    # 将归一化的结果经激活函数后返回
    return activation(bn)

def pooling_wrapper(x_input,name):
    return tf.layers.max_pooling2d(x_input,    #输入
                                  (2,2),    #池化核
                                  (2,2),    #步长
                                  name=name)

# 卷积层1_1
conv1_1=convert_wrapper(img_input, 'conv1_1',train_flag)
conv1_2=convert_wrapper(conv1_1,'conv1_2',train_flag)
conv1_3=convert_wrapper(conv1_2,'conv1_3',train_flag)
pool1=pooling_wrapper(conv1_3,'pool1')
# 卷积层2_1、2_2与第二个池化层
conv2_1=convert_wrapper(pool1,'conv2_1',train_flag)
conv2_2=convert_wrapper(conv2_1,'conv2_2',train_flag)
conv2_3=convert_wrapper(conv2_2,'conv2_3',train_flag)
pool2=pooling_wrapper(conv2_2,'pool2')   
# 卷积层3_1、3_2与第三个池化层
conv3_1=convert_wrapper(pool2,'conv3_1',train_flag)
conv3_2=convert_wrapper(conv3_1,'conv3_2',train_flag)
conv3_3=convert_wrapper(conv3_2,'conv3_3',train_flag)
pool3=pooling_wrapper(conv3_2,'pool3')


# # 初始化方法
# tf.glorot_normal_initializer
# tf.keras.initializers.he_normal
# # 优化方法
# tf.train.GradientDescentOptimizer(1e-4).minimize(loss)
# tf.train.MomentumOptimizer(learning_rate=1e-4,momentum=0.9).minimize(loss)


# 将输出的张量扁平化
flatten=tf.layers.flatten(pool3)
y_predict=tf.layers.dense(flatten,10)
# 使用交叉熵作为损失函数
loss=tf.losses.sparse_softmax_cross_entropy(y,y_predict)

# 将标签预测向量中最大的下标作为预测值，例如[0.1,0.8...0.01]，则预测为第二类
predict=tf.math.argmax(y_predict,1)
# 通过equal函数逐一比较predict，y_reshape的每一个元素
correction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correction,tf.float64))
#定义优化方法
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init=tf.global_variables_initializer()

train_steps=5000
test_steps=100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels,train_flag:True})
        if (i+1) % 500==0:
            print("第%d步：损失：%.5f，精确度：%.5f"%(i,loss_val,acc_val))
        #每训练1000次，在test数据集上进行一次测试
        if (i+1)%1000==0:
            # 定义测试集数据对象
            test_data=CifarData(test_file,False)   
            all_acc=[]
            for j in range(test_steps):
                test_batch_data,test_batch_labels=test_data.next_batch(batch_size)
                test_acc=sess.run([accuracy],feed_dict={x:test_batch_data,y:test_batch_labels,train_flag:False})
                all_acc.append(test_acc)
            # 将测得的多个准确率求均值
            test_acc_mean=np.mean(all_acc)
            print("第%d步测试集准确率%.4f"%(i,test_acc_mean))
            
            

D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Program\IDE\Anaconda\envs\TF-GPU\lib\site-packages\tensorflow\python\f

50000
第499步：损失：1.58635，精确度：0.50000
第999步：损失：1.10393，精确度：0.55000
10000
第999步测试集准确率0.2835
第1499步：损失：1.26980，精确度：0.50000
第1999步：损失：1.08659，精确度：0.60000
10000
第1999步测试集准确率0.2465
第2499步：损失：0.98081，精确度：0.75000
第2999步：损失：0.87227，精确度：0.65000
10000
第2999步测试集准确率0.1750
第3499步：损失：0.76927，精确度：0.70000
第3999步：损失：0.66512，精确度：0.80000
10000
第3999步测试集准确率0.1680
第4499步：损失：0.88021，精确度：0.65000
第4999步：损失：0.85033，精确度：0.60000
10000
第4999步测试集准确率0.1760
